In [0]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report, confusion_matrix

import pickle
import pandas as pd
import numpy as np

In [0]:
## Train multiple decision trees on train data and return object of random forest model

def Train_RandomForest(train_data, feature_list=[], No_Decision_Trees=100):
      
  train_data.dropna()
  train_data = pd.DataFrame(np.nan_to_num(np.array(train_data)), columns = train_data.columns)
  train_data['Label'] = pd.to_numeric(train_data['Label'], errors='coerce')
  train_data = train_data.dropna(subset=['Label'])
  
  train_features = train_data[feature_list]    
  train_labels = train_data["Label"]
  train_labels = train_labels.astype('int')

  rf = RandomForestRegressor(n_estimators = 100)
  rf.fit(train_features, train_labels)
  
  return rf

In [0]:
## Store trained model in a file to reuse in other codes without training again on same data

def Store_Trained_RF(RF_obj, Filepath):
  
  with open(Filepath, "wb") as file:
    pickle.dump(RF_obj, file)

In [0]:
## Load stored trained model and returns random forest model object

def Load_Trained_RF(Filepath):
  
  with open(Filepath, "rb") as file:
    RF_obj = pickle.load(file)

  return RF_obj

In [0]:
## returns predictions for test data using trained RF model 

def RF_Predict(rf_obj, test_data, feature_list=[]):
  
  test_features = test_data[feature_list]
#   print("========================================")
#   print(test_features.columns)
#   for col in test_features.columns:
#     print(col, type(col), test_features.iloc[0][col])

  predictions = rf_obj.predict(test_features)
  
  return predictions

In [0]:
def Evaluate_RF(test_data, RF_Model_FilePath, feature_list=[], threshold=0.5):
  
  test_data.dropna()
  test_data = pd.DataFrame(np.nan_to_num(np.array(test_data)),  columns = test_data.columns)
  test_data['Label'] = pd.to_numeric(test_data['Label'], errors='coerce')
  test_data = test_data.dropna(subset=['Label'])
  
  rf_obj = Load_Trained_RF(RF_Model_FilePath)
  
  predictions = RF_Predict(rf_obj, test_data, feature_list)
  
  test_labels = test_data["Label"]
  test_labels = test_labels.astype('int')
  
  for i in range(len(predictions)):
    if predictions[i] >= threshold:
      predictions[i] = 1
    else:
      predictions[i] = 0
  
  predictions = predictions.astype('int')
  
  errors = abs(predictions - test_labels)

  # Calculate mean absolute error (MAE)
  MAE = round(np.mean(errors), 2)
  
  ## Confusion Matrix and Classification Report
  Confusion_Matrix = confusion_matrix(test_labels,predictions)
  Report = classification_report(test_labels,predictions)
  
  
  return MAE, Confusion_Matrix, Report
  

In [0]:
## Execute below code for different embeddings

## Embedded_Data_Train has columns with values for text embedding of post and "Label" column has value 0 or 1
## Test_Embedding_FilePath : Where trained model will be saved
## feature_list = colnames which contains values of embeddings (exclude "Label")

Column_List = [ "Caption"]
Vector_Size = 300
Embedding_Cols = [str(i) for i in range(Vector_Size)]
Column_List.extend(Embedding_Cols)
Column_List.append("Label")

Train_Embedding_FilePath = "/content/TrainData_Word2Vec_Embeddings.csv"
Test_Embedding_FilePath = "/content/TestData_Word2Vec_Embeddings.csv"
RF_Model_FilePath = "/content/RandomForest_Train_Model.pkl"

train_data = pd.read_csv(Train_Embedding_FilePath, usecols=Column_List)
test_data = pd.read_csv(Test_Embedding_FilePath, usecols=Column_List)


## Training Phase
rf_obj = Train_RandomForest(train_data, Embedding_Cols, 100)
Store_Trained_RF(rf_obj, RF_Model_FilePath)

In [8]:
## Testing Phase
MAE, Confusion_Matrix, Report = Evaluate_RF(test_data, RF_Model_FilePath, Embedding_Cols, 0.5)

print("============ FOR WORD2VEC EMBEDDINGS ============")

print("MEAN ABSOLUTE ERROR: ", MAE)

print("\n")
print("============ CONFUSION MATRIX ===============")
print(Confusion_Matrix)

print("\n")
print("============ CLASSIFICATION REPORT ==============")
print(Report)

tn, fp, fn, tp = Confusion_Matrix.ravel()
Accuracy = (tn+tp)/(tn + fp + fn + tp)

print("Accuracy: ", Accuracy*100)

============ FOR WORD2VEC EMBEDDINGS ============
MEAN ABSOLUTE ERROR:  0.13


============ CONFUSION MATRIX ===============
[[657  76]
 [120 603]]


============ CLASSIFICATION REPORT ==============
              precision    recall  f1-score   support

           0       0.85      0.90      0.87       733
           1       0.89      0.83      0.86       723

    accuracy                           0.87      1456
   macro avg       0.87      0.87      0.87      1456
weighted avg       0.87      0.87      0.87      1456

Accuracy:  86.53846153846155


In [0]:
## GLove Embeddings

Column_List = [ "Caption_Tokens"]
Vector_Size = 300
Embedding_Cols = [str(i) for i in range(Vector_Size)]
Column_List.extend(Embedding_Cols)
Column_List.append("Label")

Train_Embedding_FilePath = "/content/TrainData_Glove_Embeddings.csv"
Test_Embedding_FilePath = "/content/TestData_Glove_Embeddings.csv"
RF_Model_FilePath = "/content/RandomForest_Glove_Train_Model.pkl"

train_data = pd.read_csv(Train_Embedding_FilePath, usecols=Column_List)
test_data = pd.read_csv(Test_Embedding_FilePath, usecols=Column_List)

## Training Phase
rf_obj = Train_RandomForest(train_data, Embedding_Cols, 100)
Store_Trained_RF(rf_obj, RF_Model_FilePath)


In [10]:
## Testing Phase
MAE, Confusion_Matrix, Report = Evaluate_RF(test_data, RF_Model_FilePath, Embedding_Cols, 0.5)

print("MEAN ABSOLUTE ERROR: ", MAE)

print("\n")
print("============ CONFUSION MATRIX ===============")
print(Confusion_Matrix)

print("\n")
print("============ CLASSIFICATION REPORT ==============")
print(Report)

tn, fp, fn, tp = Confusion_Matrix.ravel()
Accuracy = (tn+tp)/(tn + fp + fn + tp)

print("Accuracy: ", Accuracy*100)

MEAN ABSOLUTE ERROR:  0.18


============ CONFUSION MATRIX ===============
[[599 118]
 [147 592]]


============ CLASSIFICATION REPORT ==============
              precision    recall  f1-score   support

           0       0.80      0.84      0.82       717
           1       0.83      0.80      0.82       739

    accuracy                           0.82      1456
   macro avg       0.82      0.82      0.82      1456
weighted avg       0.82      0.82      0.82      1456

Accuracy:  81.79945054945054
